# Author
[Koen Aerts](https://koenaerts.ca/) @ [Mobia Technology Innovations](https://mobia.io)

[myOpenHealth](https://mobia.io/healthcare/)

# Intro
This notebook downloads the entire MIT-BIH Arrhythmia Database, which is publicly available here: https://physionet.org/physiobank/database/mitdb/

It then converts all the WFDB files into CSV files with the same name (example from 100.dat, 100.hea, 100.atr the file 100.csv is generated).

Each row in the CSV file corresponds to a single heartbeat and is generated as follows:
* Split MIT-BIH record at the R-peaks into individual heartbeat records.
* Each heartbeat record is appended with the first 40 readings of the next heartbeat record so that we include a full QRS Complex.
* Resample each heartbeat record from 360Hz to 125Hz.
* Normalize the mV readings to a 0-1 range.
* Heartbeat records longer than 187 values are discarded.
* Heartbeat records are padded with zeroes at the end until they contain exactly 187 values.
* Heartbeat classifications from the annotations is reduced to just Normal and Abnormal and appended to the end of each heartbeat record (0 is normal, 1 is abnormal). Each row then contains exactly 188 values.
* Heartbeat records without classifications are discarded.

The purpose of these CSV files is so that they can be used in training your ECG model for classifying heartbeats as either Normal or Abnormal.

# Initialize
Import dependencies.

Note that you will need to download and install the [mitdb](https://github.com/Nospoko/qrs-tutorial) library. The project contains convenience functions that make it easier to download and read [WFDB](https://physionet.org/physiotools/wfdb.shtml) compatible files. In addition, you will also need to install the [BioSPPy](https://github.com/PIA-Group/BioSPPy) library, which we use to find the R-peaks in the data.

In [ ]:
!pip install tqdm
!pip install wfdb

# Data Conversion
Read the WFDB files and convert to CSV files. Data will be split into individual heartbeats, each row consisting of exactly 187 normalized and resampled values, plus the last value with be an integer representing the classification; 0-4

In [3]:
import numpy as np
import wfdb as wf
from scipy import signal
import os
import matplotlib.pyplot as plt
from datasets import mitdb as dm
from biosppy.signals import ecg

# Load records (You might need to adjust this based on your actual data loading mechanism)
records = dm.get_records()
print('Total files: ', len(records))

# Define the relevant beats for classification
realbeats = ['N', 'V', 'F', 'S', 'Q']

class_mapping = {
    'N': 0,  # Normal
    'S': 1,  # Supraventricular
    'V': 2,  # Ventricular
    'F': 3,  # Fusion
    'Q': 4,  # Unknown
}

output_dir = 'data_ecg'
os.makedirs(os.path.join(output_dir, 'mitdb'), exist_ok=True)  # Create directory if it doesn't exist

# Initialize counters
total_annotations = 0
total_detected_beats = 0
total_removed_beats = 0
total_final_beats = 0

# Loop through each input file
for path in records:
    pathpts = path.split('/')
    fn = pathpts[-1]
    print('Loading file:', path)

    # Read in the data
    record = wf.rdsamp(path)
    annotation = wf.rdann(path, 'atr')

    # Increment total annotations
    total_annotations += len(annotation.num)

    # Print some meta information
    print('    Sampling frequency used for this record:', record[1].get('fs'))
    print('    Shape of loaded data array:', record[0].shape)
    print('    Number of loaded annotations:', len(annotation.num))

    # Get the ECG values from the file
    data = record[0].transpose()

    # Generate the classifications based on the annotations
    cat = np.array(annotation.symbol)
    rate = np.zeros_like(cat, dtype='float')
    for catid, catval in enumerate(cat):
        if catval in class_mapping:
            rate[catid] = class_mapping[catval]  # Assign class from mapping
        else:
            rate[catid] = -1  # Mark invalid or unknown beats

    rates = np.zeros_like(data[0], dtype='float')
    rates[annotation.sample] = rate

    # Process Lead II only (assuming MLII is used)
    for channelid, channel in enumerate(data):
        chname = record[1].get('sig_name')[channelid]
        if chname != "MLII":
            continue  # Skip other leads that are not Lead II

        print('    ECG channel type:', chname)

        # Find R-peaks in the ECG data
        out = ecg.ecg(signal=channel, sampling_rate=360, show=False)
        
        # Increment total detected beats
        total_detected_beats += len(out['rpeaks'])

        rpeaks = np.zeros_like(channel, dtype='float')
        rpeaks[out['rpeaks']] = 1.0

        beatstoremove = []  # Keep track of indices to remove
        beats = []  # Use a list to store valid beat segments

        # Split into individual heartbeats
        beat_segments = np.split(channel, out['rpeaks'])
        for idx, idxval in enumerate(out['rpeaks']):
            firstround = idx == 0
            lastround = idx == len(beat_segments) - 1

            # Skip first and last beat
            if firstround or lastround:
                continue

            # Get the classification value that is on or near the position of the rpeak index
            fromidx = 0 if idxval < 10 else idxval - 10
            toidx = idxval + 10
            catval = rates[fromidx:toidx].max()

            # Skip beat if there is no classification
            if catval == -1:  # Check for unknown classification
                beatstoremove.append(idx)  # Append index to list of beats to remove
                total_removed_beats += 1  # Increment removed beat count
                continue

            # Append some extra readings from next beat only if it exists
            next_beat = beat_segments[idx + 1][:40] if idx + 1 < len(beat_segments) else []

            # Combine the current beat with the next one
            beat_segment = np.append(beat_segments[idx], next_beat)

            # Normalize the readings to a 0-1 range for ML purposes
            beat_segment = (beat_segment - beat_segment.min()) / beat_segment.ptp()

            # Resample from 360Hz to 125Hz
            newsize = int((beat_segment.size * 125 / 360) + 0.5)
            beat_segment = signal.resample(beat_segment, newsize)

            # Skip records that are too long
            if beat_segment.size > 187:
                beatstoremove.append(idx)
                total_removed_beats += 1
                continue

            # Pad with zeroes if needed
            zerocount = 187 - beat_segment.size
            beat_segment = np.pad(beat_segment, (0, zerocount), 'constant', constant_values=(0.0, 0.0))

            # Append the classification to the beat data
            beat_segment = np.append(beat_segment, catval)

            # Add valid beat segment to the list
            beats.append(beat_segment)

        # Filter out beats that should be removed based on the indices
        beats = [beats[i] for i in range(len(beats)) if i not in beatstoremove]

        # Increment final beat count
        total_final_beats += len(beats)

        # Convert list of beats to a 2D NumPy array if they are all the same shape
        if beats:  # Ensure there are beats to convert
            savedata = np.array(beats, dtype=np.float32)

            # Save to CSV file
            outfn = f'{output_dir}/{fn}_{chname}.csv'
            print('    Generating ', outfn)
            with open(outfn, "w") as fin:  # Use "w" for text mode
                np.savetxt(fin, savedata, delimiter=",", fmt='%f')

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total annotations: {total_annotations}")
print(f"Total detected beats: {total_detected_beats}")
print(f"Total removed beats: {total_removed_beats}")
print(f"Final number of beats: {total_final_beats}")


Total files:  48
Loading file: data/mitdb\100
    Sampling frequency used for this record: 360
    Shape of loaded data array: (650000, 2)
    Number of loaded annotations: 2274
    ECG channel type: MLII
    Generating  data_ecg/mitdb\100_MLII.csv
Loading file: data/mitdb\101
    Sampling frequency used for this record: 360
    Shape of loaded data array: (650000, 2)
    Number of loaded annotations: 1874
    ECG channel type: MLII
    Generating  data_ecg/mitdb\101_MLII.csv
Loading file: data/mitdb\102
    Sampling frequency used for this record: 360
    Shape of loaded data array: (650000, 2)
    Number of loaded annotations: 2192
Loading file: data/mitdb\103
    Sampling frequency used for this record: 360
    Shape of loaded data array: (650000, 2)
    Number of loaded annotations: 2091
    ECG channel type: MLII
    Generating  data_ecg/mitdb\103_MLII.csv
Loading file: data/mitdb\104
    Sampling frequency used for this record: 360
    Shape of loaded data array: (650000, 2)
    